In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.callbacks as callbacks
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import numpy as np
from tqdm import tqdm

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
print('tensorflow version', tf.version.VERSION)

1 Physical GPUs, 1 Logical GPUs
tensorflow version 2.3.2


In [2]:
# URL = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
# TARGET_FILE = 'shakespeare.txt'

# path_to_file = tf.keras.utils.get_file(TARGET_FILE, URL)
# with open(path_to_file, 'rb') as f:
#   text = f.read().decode(encoding='utf-8')

In [3]:
SEQUENCE_LENGTH = 160

ds = tfds.load('tiny_shakespeare')

for example in ds['train'].take(1):
  text = example['text'].numpy().decode('utf-8')
  vocab = sorted(set(text))
  print(vocab)
  
  chars_to_index = keras.layers.experimental.preprocessing.StringLookup(vocabulary=list(vocab))
  VOCAB_SIZE = len(chars_to_index.get_vocabulary())
  print('char size: ', VOCAB_SIZE)
  
  text = tf.strings.unicode_split(text, input_encoding='UTF-8')
  print(text.shape)
  
  seq = chars_to_index(text)
  training_ds = tf.data.Dataset.from_tensor_slices(seq)
  training_ds = training_ds.batch(SEQUENCE_LENGTH + 1, drop_remainder=True)
  print(training_ds)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
char size:  67
(1003854,)
<BatchDataset shapes: (161,), types: tf.int64>


In [4]:
def seq_to_seq(seq):
  return seq[:-1], seq[1:]


BATCH_SIZE = 64
target_ds = (training_ds.map(seq_to_seq)
  .shuffle(buffer_size=1000)
  .batch(BATCH_SIZE, drop_remainder=True)
  .prefetch(tf.data.experimental.AUTOTUNE))
print(target_ds)

<PrefetchDataset shapes: ((64, 160), (64, 160)), types: (tf.int64, tf.int64)>


In [5]:
for example_inputs, example_output in target_ds.take(1):
  print(example_inputs.shape)
  print(example_output.shape)
  print(example_inputs[0])
  print(example_output[0])

(64, 160)
(64, 160)
tf.Tensor(
[60 48 45  3 53 49 44 44 52 45  3 41 54 44  3 42 61 60  3 55 54 45  3 48
 41 52 46  3 55 46  3 63 48 41 60  3 48 45  3 63 41 59  2 65 45 59 60 45
 58 44 41 65 13  3 46 55 58  3 60 48 45  3 55 60 48 45 58  3 48 41 59  3
 48 41 52 46  8  3 42 65  3 60 48 45  3 45 54 60 58 45 41 60 65  2 41 54
 44  3 47 58 41 54 60  3 55 46  3 60 48 45  3 63 48 55 52 45  3 60 41 42
 52 45 10  3 22 45  7 52 52  3 47 55  8  3 48 45  3 59 41 65 59  8  2 41
 54 44  3 59 55 63 52  3 60 48 45  3 56 55 58 60], shape=(160,), dtype=int64)
tf.Tensor(
[48 45  3 53 49 44 44 52 45  3 41 54 44  3 42 61 60  3 55 54 45  3 48 41
 52 46  3 55 46  3 63 48 41 60  3 48 45  3 63 41 59  2 65 45 59 60 45 58
 44 41 65 13  3 46 55 58  3 60 48 45  3 55 60 48 45 58  3 48 41 59  3 48
 41 52 46  8  3 42 65  3 60 48 45  3 45 54 60 58 45 41 60 65  2 41 54 44
  3 47 58 41 54 60  3 55 46  3 60 48 45  3 63 48 55 52 45  3 60 41 42 52
 45 10  3 22 45  7 52 52  3 47 55  8  3 48 45  3 59 41 65 59  8  2 41 54
 44 

In [6]:
EMBEDDING_SIZE = 256
RNN_UNITS = 1024

class GRUModel(keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = layers.Embedding(vocab_size, embedding_dim)
    self.gru = layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size, activation='softmax')

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x


def build_rnn_model():
  model =  GRUModel(
    vocab_size=len(chars_to_index.get_vocabulary()),
    embedding_dim=EMBEDDING_SIZE,
    rnn_units=RNN_UNITS)
  model.compile(optimizer=keras.optimizers.Adam(1e-3),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


keras.backend.clear_session()
model = build_rnn_model()

for input_example_batch, target_example_batch in target_ds.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary()

(64, 160, 67) # (batch_size, sequence_length, vocab_size)
Model: "gru_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  17152     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [7]:
def scheduler(epoch, lr):
  if epoch == 0:
    return 1e-3
  if epoch == 20:
    return 1e-4
  return lr


schedule_callback = callbacks.LearningRateScheduler(scheduler, verbose=True)

history = model.fit(target_ds, epochs=30,
                    callbacks=[schedule_callback])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/30
97/97 [==============================] - 5s 50ms/step - loss: 2.9821 - accuracy: 0.2326

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 2/30
97/97 [==============================] - 5s 49ms/step - loss: 2.2215 - accuracy: 0.3621

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 3/30
97/97 [==============================] - 5s 50ms/step - loss: 1.9657 - accuracy: 0.4246

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 4/30
97/97 [==============================] - 5s 49ms/step - loss: 1.7716 - accuracy: 0.4769

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 5/30
97/97 [==============================] - 5s 49ms/step - loss: 1.6317 - accuracy: 0.5143

Epoch 00006: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 6/30


In [8]:
def predict(prompt, text_length=10000):
  seq = chars_to_index(tf.strings.unicode_split(prompt, input_encoding='UTF-8')).numpy().tolist()
  states = None
  result = [i for i in seq]
  for i in tqdm(range(text_length)):
    seq_tensor = tf.convert_to_tensor([seq])
    p, state = model(seq_tensor, states=states, return_state=True, training=False)
    idx = np.argmax(p[0, -1])
    seq.append(idx)
    result.append(idx)
    states = state
    if len(seq) > SEQUENCE_LENGTH:
      seq = seq[1:]
  result_text = ''
  for idx in result:
    result_text += chars_to_index.get_vocabulary()[idx]
  return result_text


result = predict('JULIET:')
print(result)

100%|██████████| 10000/10000 [01:16<00:00, 130.98it/s]


JULIET:
And so did I, a great suspicion: but
I cannot speak a word of his own royal pires;
And so he says, in a few words for my bosom,
And be the stronger than the sea, and let him
show your father's blood.

LUCIO:

ISABELLA:
Alas, poor soul!

ANGELO:
What is the matter?

MENENIUS:
I will be satisfied.

Provost:
I would they were a beggar to his children's loss,
And see how he hath something in the state
Of the deep issues; and therefore have you all
from the heavens have strong and tell him with a viserable and
The very blood of my own life did stand and
see him out.

DUKE VINCENTIO:
The ship spirits in the sun.

PAULINA:
There is no other way
Unto the stroke of fortune and the sun
And with thy state and in the house of York
May she had left us out another and my father,
And therefore has a serious man; and then
I came to thee and the present at the people,
And that the sun hath set the sea for the world,
That they are gone to keep your part in justice,
That is not heaven and enter t